# GKY (2020) Replication: Part 4 - Model Training & Evaluation

##### This is the main workhorse notebook for the GKY replication project. It implements the complete training, tuning, and out-of-sample (OOS) prediction loop.

##### It is structured around an object-oriented design:
##### 1.  **A `BaseModel` Class:** This class acts as a generic "engine" that handles the entire evaluation process:
##### - It manages the annual looping through training, validation, and testing periods.
##### - It uses `GridSearchCV` with `PredefinedSplit` to correctly tune hyperparameters on the validation set without any data leakage.
##### - It collects all OOS predictions and calculates the final R²_OOS metric as defined in the paper.

##### 2.  **Specific Model Subclasses:** Each machine learning model (e.g., PCR, Random Forest) is a simple class that inherits from `BaseModel`. Its only job is to provide its specific scikit-learn estimator and hyperparameter grid, making the code clean, modular, and easy to extend.

##### 3.  **Execution Block:** The final section instantiates each model object and runs the full evaluation, collecting the results to replicate Table 1.


In [1]:
import pandas as pd
import numpy as np

import os
import gc

from tqdm import tqdm

# --- Scikit-learn Imports ---
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression, HuberRegressor, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [2]:
# Define Constants

FIRST_OOS_YEAR = 1987
LAST_OOS_YEAR = 2016
TRAINING_START_YEAR = 1957
VALIDATION_LENGTH_YEARS = 12


In [3]:
# --- Part 1: Setup and Framework ---
print("--- Part 1: Setup and Framework ---")

# Define file paths and constants
INPUT_DIR = './data'
INPUT_PATH = os.path.join(INPUT_DIR, 'gky_final_data.parquet')

# Load the final analysis-ready dataset
print(f"Loading final dataset from '{INPUT_PATH}'...")
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError("Final data file not found. Please run 01 and 02 notebooks first.")
gky_final_data = pd.read_parquet(INPUT_PATH)
gky_final_data['month'] = pd.to_datetime(gky_final_data['month'])
print(f"Data loaded successfully. Shape: {gky_final_data.shape}")


--- Part 1: Setup and Framework ---
Loading final dataset from './data/gky_final_data.parquet'...
Data loaded successfully. Shape: (2851604, 923)


In [4]:
# Create the evaluation periods DataFrame
periods = []
for year in range(FIRST_OOS_YEAR, LAST_OOS_YEAR + 1):
    periods.append({
        'oos_year': year,
        'train_start': TRAINING_START_YEAR,
        'train_end': year - VALIDATION_LENGTH_YEARS - 1,
        'valid_start': year - VALIDATION_LENGTH_YEARS,
        'valid_end': year - 1,
    })
evaluation_periods_df = pd.DataFrame(periods)
print("Evaluation periods framework created.")
print(evaluation_periods_df.head(3))

# Define predictor columns once
ID_COLS = ['permno', 'month', 'ret_excess', 'mktcap_lag']
PREDICTOR_COLS = [col for col in gky_final_data.columns if col not in ID_COLS]
print(f"Identified {len(PREDICTOR_COLS)} predictor columns.")


Evaluation periods framework created.
   oos_year  train_start  train_end  valid_start  valid_end
0      1987         1957       1974         1975       1986
1      1988         1957       1975         1976       1987
2      1989         1957       1976         1977       1988
Identified 919 predictor columns.


In [5]:
# --- Part 2: The Core Modeling Engine (`BaseModel` Class) ---
print("\n--- Part 2: Defining the Core `BaseModel` Engine ---")

class BaseModel:
    """
    A generic base class for handling the GKY (2020) evaluation methodology.
    
    This class orchestrates the annual training, validation, and testing loop,
    performs hyperparameter tuning using GridSearchCV with a PredefinedSplit,
    and calculates the final out-of-sample R-squared.
    """
    def __init__(self, model_name: str, estimator: BaseEstimator, param_grid: dict, predictor_subset: list = None):
        self.model_name = model_name
        self.estimator = estimator
        self.param_grid = param_grid
        self.predictor_subset = predictor_subset if predictor_subset else PREDICTOR_COLS
        self.all_oos_predictions = []
        
    def _train_and_predict_one_year(self, full_df: pd.DataFrame, period: pd.Series) -> pd.DataFrame:
        """
        Handles the model training, tuning, and prediction for a single OOS year.
        """
        # 1. Slice data into train, validation, and OOS test sets
        train_df = full_df[
            (full_df['month'].dt.year >= period.train_start) & 
            (full_df['month'].dt.year <= period.train_end)
        ]
        valid_df = full_df[
            (full_df['month'].dt.year >= period.valid_start) & 
            (full_df['month'].dt.year <= period.valid_end)
        ]
        oos_df = full_df[full_df['month'].dt.year == period.oos_year]

        # 2. Prepare data for scikit-learn
        X_train, y_train = train_df[self.predictor_subset], train_df['ret_excess']
        X_valid, y_valid = valid_df[self.predictor_subset], valid_df['ret_excess']
        X_oos,   y_oos   = oos_df[self.predictor_subset],   oos_df['ret_excess']
        
        X_tune = pd.concat([X_train, X_valid], ignore_index=True)
        y_tune = pd.concat([y_train, y_valid], ignore_index=True)

        # 3. Set up PredefinedSplit for GridSearchCV
        # -1 for training samples, 0 for validation samples
        split_index = [-1] * len(X_train) + [0] * len(X_valid)
        pds = PredefinedSplit(test_fold=split_index)

        # 4. Tune hyperparameters
        tuner = GridSearchCV(
            estimator=self.estimator,
            param_grid=self.param_grid,
            scoring='neg_mean_squared_error',
            cv=pds,
            n_jobs=-1,  # Use all available CPU cores
            verbose=0   # Set to 1 or 2 for detailed tuning logs
        )
        tuner.fit(X_tune, y_tune)
        
        # 5. Make predictions on the OOS set
        best_model = tuner.best_estimator_
        predictions = best_model.predict(X_oos)

        # 6. Store predictions with identifiers
        oos_results = oos_df[ID_COLS].copy()
        oos_results['prediction'] = predictions
        
        return oos_results

    def _calculate_oos_r2(self) -> float:
        """
        Calculates the OOS R-squared based on the paper's formula.
        """
        if not self.all_oos_predictions:
            raise ValueError("No OOS predictions found. `run_full_evaluation` must be called first.")
        
        preds_df = pd.concat(self.all_oos_predictions, ignore_index=True)
        
        sse = ((preds_df['ret_excess'] - preds_df['prediction']) ** 2).sum()
        sst = (preds_df['ret_excess'] ** 2).sum() # Note: Denominator is not variance
        
        if sst == 0: return -np.inf # Avoid division by zero
        
        r2_oos = 1 - (sse / sst)
        return r2_oos * 100 # Return as a percentage

    def run_full_evaluation(self, full_df: pd.DataFrame, periods_df: pd.DataFrame) -> tuple[float, pd.DataFrame]:
        """
        Executes the full evaluation loop over all specified periods.
        """
        print(f"\n--- Starting Full Evaluation for: {self.model_name} ---")
        self.all_oos_predictions = [] # Reset predictions
        
        for _, period in tqdm(periods_df.iterrows(), total=len(periods_df), desc=self.model_name):
            try:
                oos_preds_for_year = self._train_and_predict_one_year(full_df, period)
                self.all_oos_predictions.append(oos_preds_for_year)
            except Exception as e:
                print(f"ERROR during {self.model_name} evaluation for OOS year {period.oos_year}: {e}")
            gc.collect() # Clean memory after each intensive year
            
        final_r2 = self._calculate_oos_r2()
        print(f"--- Evaluation Complete for: {self.model_name} | Final OOS R² = {final_r2:.4f}% ---")
        
        return final_r2, pd.concat(self.all_oos_predictions, ignore_index=True)

print("BaseModel class defined successfully.")


--- Part 2: Defining the Core `BaseModel` Engine ---
BaseModel class defined successfully.


In [6]:
# --- Custom Estimator for GKY's Huber Loss Methodology ---
class GKYHuberRegressor(BaseEstimator, RegressorMixin):
    """
    A custom scikit-learn estimator that faithfully replicates the GKY (2020)
    methodology for setting the Huber loss epsilon (ξ).
    
    The paper specifies epsilon as the 99.9% quantile of the absolute residuals.
    This estimator performs a two-stage fit:
    1. A preliminary OLS fit is used to generate residuals.
    2. The 99.9% quantile of these absolute residuals is calculated.
    3. The final HuberRegressor is fit using this dynamically determined epsilon.
    """
    def __init__(self, max_iter=500):
        # max_iter is kept as a practical parameter to prevent convergence issues.
        self.max_iter = max_iter
        self.epsilon_ = None # Will store the calculated epsilon
        self.final_huber_ = None # Will store the final fitted model

    def fit(self, X, y):
        # Stage 1: Preliminary fit to get residuals
        # Using a simple OLS is fast and effective for this purpose.
        prelim_model = LinearRegression(n_jobs=-1).fit(X, y)
        residuals = y - prelim_model.predict(X)
        
        # Stage 2: Calculate the dynamic epsilon as per the paper
        calculated_epsilon = np.quantile(np.abs(residuals), 0.999)
        
        # Stage 3: Adapt to scikit-learn's constraint (epsilon >= 1.0)
        # This is the most faithful adaptation possible.
        self.epsilon_ = max(1.0, calculated_epsilon)
        
        # Stage 4: Fit the final HuberRegressor with the correct, valid epsilon
        self.final_huber_ = HuberRegressor(
            epsilon=self.epsilon_, 
            max_iter=self.max_iter
        ).fit(X, y)
        
        return self

    def predict(self, X):
        if self.final_huber_ is None:
            raise RuntimeError("You must fit the model before making predictions.")
        return self.final_huber_.predict(X)

print("Custom GKYHuberRegressor class defined successfully.")

Custom GKYHuberRegressor class defined successfully.


In [7]:
# --- Part 3: Model Definitions (Subclasses) ---
print("\n--- Part 3: Defining Specific Model Implementations ---")

# --- Linear Models ---
class OLS3HModel(BaseModel):
    def __init__(self):
        predictors = [
            'characteristic_mvel1_x_macro_intercept', 
            'characteristic_bm_x_macro_intercept', 
            'characteristic_mom12m_x_macro_intercept'
        ]
        # This pipeline now uses our custom, faithful GKYHuberRegressor.
        # Scaling is still critical for performance and convergence.
        estimator = Pipeline([
            ('scaler', StandardScaler()),
            ('reg', GKYHuberRegressor(max_iter=500))
        ])
        param_grid = {} # No hyperparameters to tune
        super().__init__(model_name="OLS-3+H", estimator=estimator, param_grid=param_grid, predictor_subset=predictors)

class OLSHModel(BaseModel):
    def __init__(self):
        # Using the custom GKYHuberRegressor for the full model as well.
        estimator = Pipeline([
            ('scaler', StandardScaler()),
            ('reg', GKYHuberRegressor(max_iter=500))
        ])
        param_grid = {} # No hyperparameters to tune
        super().__init__(model_name="OLS+H", estimator=estimator, param_grid=param_grid)
        
class PCRModel(BaseModel):
    def __init__(self):
        estimator = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('reg', LinearRegression())
        ])
        # Grid based on GKY Figure 3, where K is often 20-40.
        param_grid = {'pca__n_components': [10, 20, 30, 40, 50, 60]}
        super().__init__(model_name="PCR", estimator=estimator, param_grid=param_grid)

class ENetHModel(BaseModel):
    def __init__(self):
        # NOTE: scikit-learn's ElasticNet does not support Huber loss directly.
        # This implementation uses the standard ElasticNet as the closest available
        # off-the-shelf model, as allowed by the assignment. A full replication
        # of the "+H" would require a different library.
        estimator = Pipeline([
            ('scaler', StandardScaler()),
            ('reg', ElasticNet(l1_ratio=0.5, random_state=42, max_iter=1000))
        ])
        # Grid based on GKY Table A.5
        param_grid = {'reg__alpha': np.logspace(-4, -1, 4)}
        super().__init__(model_name="ENet+H", estimator=estimator, param_grid=param_grid)

# --- Tree-Based Models ---
class RFModel(BaseModel):
    def __init__(self):
        estimator = RandomForestRegressor(
            n_estimators=300,
            random_state=42,
            n_jobs=-1 # Use all cores for tree building
        )
        # Grid based on GKY Table A.5 and TidyFinance post
        param_grid = {
            'max_features': [10, 20, 30, 50],
            'min_samples_leaf': [5000, 10000] # Controls tree depth
        }
        super().__init__(model_name="RF", estimator=estimator, param_grid=param_grid)

# --- Neural Network Placeholders ---
# These will be implemented using a deep learning library in the next step.
class NN2Model(BaseModel):
     def __init__(self):
        # This is a placeholder and will not run correctly until implemented.
        super().__init__(model_name="NN2", estimator=None, param_grid={})

class NN4Model(BaseModel):
     def __init__(self):
        # This is a placeholder and will not run correctly until implemented.
        super().__init__(model_name="NN4", estimator=None, param_grid={})

print("All model classes defined successfully.")



--- Part 3: Defining Specific Model Implementations ---
All model classes defined successfully.


In [9]:
# --- Part 4: The Execution Block ---
print("\n--- Part 4: Running the Evaluation ---")

# NOTE: This block will take a significant amount of time to run,
# especially for models with large hyperparameter grids.
# It is recommended to run one or two models at a time to test.

if __name__ == "__main__":
    
    # Instantiate all models we want to evaluate for Table 1
    models_to_run = {
        "OLS+H": OLSHModel(),
        "OLS-3+H": OLS3HModel(),
        # "PCR": PCRModel(),
        # "ENet+H": ENetHModel(),
        # "RF": RFModel(),
        # "NN2": NN2Model(), # Uncomment when implemented
        # "NN4": NN4Model(), # Uncomment when implemented
    }
    
    final_results = {}

    for name, model_obj in models_to_run.items():
        try:
            r2_score, oos_preds = model_obj.run_full_evaluation(gky_final_data, evaluation_periods_df)
            final_results[name] = r2_score
            
            # Optional: Save the detailed predictions for later analysis (e.g., Figure 9)
            # preds_output_path = os.path.join(INPUT_DIR, f'preds_{name.replace("+", "")}.parquet')
            # oos_preds.to_parquet(preds_output_path, index=False)
            # print(f"Saved predictions for {name} to {preds_output_path}")

        except Exception as e:
            print(f"!!! FAILED EVALUATION FOR {name}: {e} !!!")
            final_results[name] = 'Failed'
    
    # --- Display Final Table 1 Results ---
    print("\n\n--- FINAL OOS R² RESULTS (Replication of Table 1) ---\n")
    
    results_df = pd.DataFrame.from_dict(final_results, orient='index', columns=['R2_OOS (%)'])
    results_df.index.name = "Model"
    
    # Format the DataFrame to match the paper's style
    table1_output = pd.DataFrame(index=['All'])
    for model_name in ["OLS+H", "OLS-3+H", "PCR", "ENet+H", "RF"]: # Add NNs here later
        if model_name in results_df.index:
            table1_output[model_name] = f"{results_df.loc[model_name, 'R2_OOS (%)']:.2f}"
        else:
            table1_output[model_name] = "N/A"
            
    print(table1_output)
    print("\nReplication of scikit-learn based models complete.")
    


--- Part 4: Running the Evaluation ---

--- Starting Full Evaluation for: OLS+H ---


OLS+H:   3%|▎         | 1/30 [01:46<51:14, 106.00s/it]

ERROR during OLS+H evaluation for OOS year 1987: 
All the 1 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/tmp/ipykernel_1927503/449136647.py", line 32, in fit
  File 

OLS+H:   7%|▋         | 2/30 [03:12<44:07, 94.57s/it] 

ERROR during OLS+H evaluation for OOS year 1988: 
All the 1 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/tmp/ipykernel_1927503/449136647.py", line 32, in fit
  File 

OLS+H:  10%|█         | 3/30 [04:36<40:21, 89.69s/it]

ERROR during OLS+H evaluation for OOS year 1989: 
All the 1 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/tmp/ipykernel_1927503/449136647.py", line 32, in fit
  File 

OLS+H:  13%|█▎        | 4/30 [06:28<42:44, 98.65s/it]

ERROR during OLS+H evaluation for OOS year 1990: 
All the 1 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/data2/home/lugia10/.conda/envs/gkx_env/lib/python3.10/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/tmp/ipykernel_1927503/449136647.py", line 32, in fit
  File 